<a href="https://colab.research.google.com/github/souvikmajumder26/Any-Face-Clustering/blob/main/Detailed_Project_Explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Any Face Clustering:**

> ### Step 1: *Runtime > Change Runtime Type > Hardware Accelerator > GPU > Save*
> 
> ### Step 2: *Run all cells sequentially to visualize the mechanism and output of the project.*

---
## **Encoding faces**
---

### *Installing & Importing required packages*

In [5]:
!pip install face_recognition

In [8]:
import cv2
from imutils import paths
import face_recognition
import glob
import cv2
import os

### *Recognizing and Encoding the faces numerically, each with a 128-D feature vector generated by a Neural Network*

In [ ]:
# UPLOAD the images here to test

from google.colab import files
upload = files.upload()

In [ ]:
data = []

#path = "/content/drive/MyDrive/Colab Data/Personal_Projects/Any_Face_Clustering/unlabelled_test_images/*.*"

#for file in glob.glob(path):
for file in upload:
  print("[INFO] processing image {}".format(file))
  image = cv2.imread(file)
  rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  boxes = face_recognition.face_locations(rgb, model = "cnn")
  # computing the facial embedding for the face
  encodings = face_recognition.face_encodings(rgb, boxes)
  # building a dictionary of the image path, bounding box location, and encodings for the current image
  d = [{"imagePath": file, "loc": box, "encoding": enc} for (box, enc) in zip(boxes, encodings)]
  data.extend(d)

  #cv2_imshow(image)
  #cv2_imshow(rgb)
  #print(boxes)
  #print(encodings)

---
## **Clustering the faces**
---

### *Installing & Importing required packages*

In [ ]:
from google.colab.patches import cv2_imshow
from sklearn.cluster import DBSCAN
from imutils import build_montages
import numpy as np

### *Preparing the Data and Face Encodings for clustering*

In [ ]:
# converting the data into a numpy array
np_data = np.array(data)
# extracting the 128-d facial encodings and placing them in a list
np_encodings = [item["encoding"] for item in np_data]

### *Clustering the Face Encodings using DBSCAN*

In [ ]:
print("[INFO] clustering...")
# initializing the clustering object
cluster = DBSCAN(metric = "euclidean", n_jobs = -1)
# fitting the clustering model on the encoded data
cluster.fit(np_encodings)

### *Determining the number of unique faces recognized from the input data*

In [ ]:
labelIDs = np.unique(cluster.labels_)
numUniqueFaces = len(np.where(labelIDs > -1)[0])
print("[INFO] No. of unique faces: {}".format(numUniqueFaces))
print("[INFO] Label IDs: {}, where [-1] refers to unidentified faces or outliers".format(labelIDs))

### *Visualizing the images belonging to each unique labelID after clustering the faces*

In [ ]:
# loop over the unique face integers
for labelID in labelIDs:
  # find all indexes into the 'pkl_data' array that belong to the current label ID, then randomly sample a maximum of 15 indexes from the set
  print("[INFO] faces for face ID: {}".format(labelID))
  idxs = np.where(cluster.labels_ == labelID)[0]
  idxs = np.random.choice(idxs, size = min(15, len(idxs)), replace = False)
  # initialize the list of faces to include in the montage
  faces = []
  
  for i in idxs:
    # load the input image and extract the face ROI
    current_image = cv2.imread(np_data[i]["imagePath"])
    (top, right, bottom, left) = np_data[i]["loc"]
    current_face = current_image[top:bottom, left:right]
    # force resize the face ROI to 96x96 and then add it to the faces montage list
    current_face = cv2.resize(current_face, (96, 96))
    faces.append(current_face)
    
  # create a montage using 96x96 "tiles" with 5 rows and 5 columns
  montage = build_montages(faces, (96, 96), (3, 3))[0]
  
  # show the output montage
  # current_title = "Face ID #{}".format(labelID)
  # current_title = "Unknown Faces" if labelID == -1 else current_title
  cv2_imshow(montage)